In [1]:
import requests
import re
import os
from dotenv import load_dotenv
load_dotenv('../config/enviroment.env')
import pandas as pd
import json

In [2]:
# URL BASE PARA REALIZAR LAS CONSULTAS A LA API
URL = 'http://api.musixmatch.com/ws/1.1/'
num_search = 10
# OBTENER LA key PARA HABILITAR EL SERVICIO
API_KEY = f'apikey={os.getenv("API_KEY")}'

In [3]:
# Artistas TOP en Mexico
def search_topArtists():
    page_size = '80' # As default
    url_topArtists = f'{URL}chart.artists.get?{API_KEY}&page_size={page_size}&country=MX'
    respons = requests.get(url_topArtists)
    if respons.status_code == 200:
        content = [l['artist'] for l in respons.json()['message']['body']['artist_list']]
        lists_topArtists = [(l['artist_id'],l['artist_name'],l['artist_country'],l['artist_rating'])  for l in content]
        pd_topArtists = pd.DataFrame(lists_topArtists, columns = ['artist-ID','Nombre','Pais','Rating'])
        return pd_topArtists
    else:
        return f'No hay respuesta del servidor'

In [4]:
# Canciones top en México
def search_topTracks():
    page_size = '80'
    url_topTracks = f'{URL}chart.tracks.get?{API_KEY}&page_size={page_size}&country=MX&chart_name=hot&f_has_lyrics=1'
    respons = requests.get(url_topTracks)
    if respons.status_code == 200:
        content = [l['track'] for l in respons.json()['message']['body']['track_list']]
        list_topTracks = [(l['track_id'],l['album_id'],l['artist_id'],l['track_name'],l['album_name'],l['artist_name'],l['track_rating'])  for l in content]
        pd_topTracks = pd.DataFrame(list_topTracks, columns=['track-ID','album-ID','artist-ID','Nombre','Album','Artista','Rating'])
        return pd_topTracks
    else:
        return f'No hay respuesta del servidor'


In [5]:
# Albumes del artista
def search_ArtistAlbum(artist_id):
    page_size = '100'
    url_albumArtist = f'{URL}artist.albums.get?{API_KEY}&artist_id={artist_id}&s_release_date=desc&page_size={page_size}&g_album_name=1'
    respons = requests.get(url_albumArtist)
    if respons.status_code == 200:
        content = [l['album'] for l in respons.json()['message']['body']['album_list']]
        album_list = [(l['album_id'], l['artist_id'], l['album_name'],l['album_release_date'], l['album_rating']) for l in content]
        pd_albumes = pd.DataFrame(album_list, columns=['album-ID','artist-ID','Nombre','Fec-Release','Rating'])
        return pd_albumes

In [6]:
# Obtiene lista de canciones por album
def get_albumTracks(album_id):
    url_albumTracks = f'{URL}album.tracks.get?{API_KEY}&album_id={album_id}&page=1'
    respons = requests.get(url_albumTracks)
    if respons.status_code == 200:
        content = [l['track'] for l in respons.json()['message']['body']['track_list']]
        track_list = [(l['track_id'], l['album_id'], l['artist_id'], l['track_name'], l['track_rating'], l['has_lyrics']) for l in content]
        pd_tracks = pd.DataFrame(track_list, columns = ['track-ID','album-ID','artist-ID','Nombre','Rating','con-Letra'])
    return pd_tracks

In [8]:
#PROGRAMA PRINCIPAL
#Artistas TOP actualmente en MX
artists_top = search_topArtists()
artists_top.sort_values(by=['Rating'], inplace = True, ascending = False)
artists_top.to_csv('../output/artists_top.csv', index = False)

#Cancciones TOP consultadas en las últimas 2 horas
tracks_top = search_topTracks()
tracks_top.sort_values(by=['Rating'], inplace = True, ascending = False)
tracks_top.to_csv('../output/tracks_top.csv', index = False)

#Obtiene todos los discos de los artitas top en MX
for art in range(len(artists_top.index)-1):
    alb = search_ArtistAlbum(artists_top['artist-ID'].values[art])
    if art == 0:
        pd_albumes = alb
    else:
        pd_albumes = pd.concat([pd_albumes, alb], ignore_index=True)
        

        
pd_albumes.to_csv('../output/album_artists.csv', index = False)

#Obtiene las pistas de los discos de los artistas top, solo los primeros 4 discos
for tr in range(len(pd_albumes.index)-1):
    tracks = get_albumTracks(pd_albumes['album-ID'].values[tr])
    if tr == 0:
        pd_tracks = tracks
    else:
        pd_tracks = pd.concat([pd_tracks, tracks], ignore_index = False)
    

pd_tracks.to_csv('../output/tracks_collect.csv', index = False)